In [1]:
import numpy as np

In [2]:
a = np.arange(24).reshape((2,3,4))
a

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [15]:
a.reshape((a.shape[0], -1))

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]])

In [5]:
help(np.flatten)

AttributeError: module 'numpy' has no attribute 'flatten'